<a href="https://colab.research.google.com/github/samueljbp/HackatonFase5/blob/main/Executando%20o%20modelo%20no%20video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection

In [ ]:
import logging
# Configuração do Logger
# logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info(f"SYSTEM:: ===== Iniciando programa =====")

In [ ]:
# Mensagem final
# print(f"Processamento do vídeo concluído!")
logging.info(f"DETECTION:: Fim do processo de detecção")
# print(f"Total de detecções: {total_detections}")
logging.info(f"DETECTION:: Total de detecções: {total_detections}")
# print(f"Frames salvos em: {output_frames_dir}")

# Mail

In [ ]:
# Definindo variáveis principais
# IMAGENS = ["imagem1", "imagem2"]
IMAGENS_DIR = output_frames_dir
IMAGENS = [f"imagem_{i+1}" for i in range(saved_frame_idx)]

QTD = len(IMAGENS)
VIDEO = os.path.basename(video_path)
DESTINATARIOS = ["email1@gmail.com", "email2@gmail.com"]
REMETENTE = "fakevisionguard@gmail.com"

In [ ]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.utils import parseaddr
from sys import exit

In [ ]:
# Função para validar email
def validar_email(email):
    nome, endereco = parseaddr(email)
    if '@' in endereco and '.' in endereco.split('@')[-1]:
        return True
    return False

# Função para carregar a senha do arquivo 'secret.txt'
def carregar_senha():
    # try:
    #     with open("secret.txt", "r") as file:
    #         password = file.read().strip()
    #         if not password:
    #             logging.critical("MAIL:: Senha não encontrada")
    #             exit(1)
    #         return password
    # except FileNotFoundError:
    #     logging.critical("MAIL:: O arquivo 'secret.txt' não foi encontrado.")
    #     exit(1)
    return "wixb qyrv qvkc qqwh"

# Função para enviar e-mail
def enviar_email(sender_email, receiver_email, qtd, video, password):
    # Criando parametros da mensagem
    # message = MIMEMultipart("alternative") # Para enviar apenas texto
    message = MIMEMultipart("related") # Para enviar imagens
    message["Subject"] = "Alerta Segurança"
    message["From"] = sender_email
    message["To"] = receiver_email

    # Tratamento de exceção para qtd e video
    if not qtd or not video:
        logging.warning("MAIL:: Informações faltando. QTD ou VIDEO não foi fornecido.")
        qtd = "desconhecido"
        video = "desconhecido"

    # Criando corpo do email com Imagens
    imagens_html = ""
    for i in range(min(len(IMAGENS), 10)):
        imagens_html += f'<img src="cid:{IMAGENS[i]}" alt="Imagem de Alerta" width="250" height="250">'


    # Criando o corpo do email em HTML
    html = f"""
    <html>
        <body style="font-family: Arial, sans-serif; background-color: #f4f4f4; color: #333; padding: 20px;">
            <table role="presentation" style="width: 100%; border: 1px solid #ddd; background-color: #fff; padding: 15px;">
                <tr>
                    <td>
                        <h2 style="color: #d9534f;">Alerta de Segurança</h2>
                        <p>Foram detectados <strong>{qtd}</strong> objetos perigosos durante o escaneamento do vídeo <strong>{video}</strong>.</p>
                        <p><a href="#" style="color: #007bff; text-decoration: none;">Acesse as imagens</a></p>
                        {imagens_html}
                        <br />
                        <footer style="font-size: 0.9em; color: #777;">
                            <b>Vision Guard Seguradora LTDA.</b><br />
                            Todos os direitos reservados.
                        </footer>
                    </td>
                </tr>
            </table>
        </body>
    </html>
    """

    # Convertendo o HTML para objetos MIMEText e acoplando a mensagem
    body = MIMEText(html, "html")
    message.attach(body)
    logging.info("MAIL:: Corpo do email criado com sucesso")


    for i, img_id in enumerate(IMAGENS[:10]):
        img_path = os.path.join(output_frames_dir, f"imagem_{i+1}.jpg")
        with open(img_path, "rb") as img_file:
            img = MIMEImage(img_file.read())
            img.add_header('Content-ID', f'<{img_id}>')
            message.attach(img)
            logging.info(f"MAIL:: Imagem {img_id} adicionada ao e-mail")



    # Criando uma conexão com o servidor e enviando o email
    try:
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            logging.info("MAIL:: Conectando ao servido SMTP")
            server.login(sender_email, password)
            logging.info("MAIL:: Logado no servidor com sucesso")
            server.sendmail(sender_email, receiver_email, message.as_string())
            logging.info(f"MAIL:: Email enviado com sucesso para {receiver_email}")
    except smtplib.SMTPException as e:
        logging.error(f"MAIL:: Erro ao enviar o email: {str(e)}")
    except Exception as e:
        logging.critical(f"MAIL:: Erro inesperado: {str(e)}")

In [ ]:
# Validando os emails
if not validar_email(REMETENTE):
    print(f"Endereço de e-mail do remetente inválido: {REMETENTE}")
    exit(1)

DESTINATARIOS_VALIDOS = []
for email in DESTINATARIOS:
    if validar_email(email):
        DESTINATARIOS_VALIDOS.append(email)
    else:
        print(f"Endereço de e-mail inválido: {email}")

if not DESTINATARIOS_VALIDOS:
    print("Nenhum destinatário válido encontrado.")
    exit(1)

# Carregando a senha do arquivo 'secret.txt'
PASSWORD = carregar_senha()

# Enviando os e-mails
sender_email = REMETENTE
for receiver_email in DESTINATARIOS_VALIDOS:
    enviar_email(sender_email, receiver_email, QTD, VIDEO, PASSWORD)

logging.info(f"SYSTEM:: ===== Finalizando programa =====")